We are going to start creating a series for training.

y[0] = x[0]

y[1] = x[1]

y[2] = x[0]+x[1]


We are going to create different trainning sequences with different startup variables

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn

In [50]:
num_seqs=7
seq_len=30
seqs_x=np.array([]).reshape(-1,seq_len)
seqs_y=np.array([]).reshape(-1,seq_len)
for i in xrange(num_seqs):
    x=np.empty((0),dtype=np.float64)
    for j in xrange(seq_len):
        x=np.append(x,np.float64(np.random.randint(0,50)))

    y=np.empty_like(x)
    y[0]=x[0]
    y[1]=x[1]
    for j in xrange(2,len(x)):
        y[j]=x[j]+x[j-1]
    seqs_x=np.concatenate((seqs_x, np.reshape(x,(1,-1))))
    seqs_y=np.concatenate((seqs_y, np.reshape(y,(1,-1))))
    
    

print "Sequences X:"
print seqs_x
print "Sequences Y:"
print seqs_y

Sequences X:
[[ 3.  6.  2. 49. 24.  8. 40.  6. 28. 46. 33. 46. 30. 36. 24.  7. 10. 17.
   9.  2.  5. 39. 36. 37. 33. 29. 10. 33. 42. 42.]
 [43.  5. 28. 12. 35. 27. 27. 27.  0. 11. 12. 26. 36. 35. 34. 11. 40. 49.
   0. 39. 23.  9.  2. 37. 39. 38.  9.  1. 28. 22.]
 [17. 21. 38. 15.  1. 20. 12. 15. 19. 48. 43. 38. 41. 49. 21.  2. 12. 44.
  40. 27.  9. 32. 49. 16. 49. 36. 27. 26. 15. 38.]
 [49. 33. 47. 19. 35. 38. 21. 42.  9. 33. 17. 35. 11.  7. 34. 35. 32.  2.
  11. 23. 32. 30. 14. 31. 39. 23. 32. 46. 47. 29.]
 [ 0. 21. 21. 21. 32. 16. 48. 28. 20. 28. 23. 22. 41.  7. 37.  3. 44. 49.
  14. 12. 43. 19. 23. 12. 49. 16. 31. 11. 38. 10.]
 [ 8. 24. 31. 46. 48. 10. 43.  3.  4. 13. 49.  1. 35. 24. 40. 20.  0. 24.
  48. 47.  0. 17. 11. 33. 30. 16. 41. 11. 39. 23.]
 [28. 15. 17.  4. 40. 28. 34. 39. 18. 47. 23. 33. 32. 46. 19. 44. 28. 35.
  33. 17. 34. 36. 42. 47. 41.  6. 29. 16.  4. 38.]]
Sequences Y:
[[ 3.  6.  8. 51. 73. 32. 48. 46. 34. 74. 79. 79. 76. 66. 60. 31. 17. 27.
  26. 11.  7. 44. 75. 73

In [51]:
th = np.int(num_seqs*0.7)
train_x= seqs_x[:th]
test_x = seqs_x[th:]
train_y= seqs_y[:th]
test_y = seqs_y[th:]

In [52]:
lstm_size = 10
num_features = 1

In [53]:
tf.reset_default_graph()

#The input data is going to be [batch_size]
x_data = tf.placeholder(tf.float32, [None, 1],"x_data")

#The output data is going to be [batch_size, 1]
y_real = tf.placeholder(tf.float32, [None, 1],"y_real")

lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size, state_is_tuple=True,name="LSTM_cell")
c_in=tf.placeholder(tf.float32, [None, lstm_size],name="c_in")
h_in=tf.placeholder(tf.float32, [None, lstm_size],name="h_in")
                    
init_state = tf.contrib.rnn.LSTMStateTuple( c=c_in, h=h_in)


# The value of state is updated after processing each batch of words.
output_rnn, out_state = lstm(x_data, init_state)

W_out = tf.get_variable("Wout",initializer=tf.random_normal([lstm_size,1]))
b_out = tf.get_variable("b_out",initializer=tf.random_normal([1]))

# The LSTM output can be used to make next word predictions
out = tf.matmul(output_rnn, W_out) + b_out

loss = tf.losses.mean_squared_error(y_real,out)
#train_op = tf.train.AdamOptimizer(learning_rate=0.0002).minimize(loss)

optimizer = tf.train.AdamOptimizer(learning_rate=0.0002)
gvs = optimizer.compute_gradients(loss)
capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
train_op = optimizer.apply_gradients(capped_gvs)

sess=tf.Session()
sess.run(tf.global_variables_initializer())

# We save the graph to analyze it in Tensorboard
train_writer = tf.summary.FileWriter('./graphtf',sess.graph)

Le'ts try with an example, yust one sample from the list

In [54]:
np_c = np.zeros((1, lstm_size),np.float32)
np_h = np.zeros((1, lstm_size),np.float32)

o = sess.run([out], feed_dict={x_data: train_x[0:1,0].reshape(-1,1), 
                                      y_real:train_y[0:1,0].reshape(-1,1),
                                      c_in:np_c, h_in:np_h})

print "The estimated output without training is:"
print np.squeeze(o)
print "But it should be:"
print train_y[1,0]

The estimated output without training is:
0.17150217
But it should be:
43.0


Let's train the model

In [55]:
max_epocs=10000
for j in xrange(max_epocs):
    np_c = np.zeros((train_x.shape[0], lstm_size),np.float32)
    np_h = np.zeros((train_x.shape[0], lstm_size),np.float32)
    for i in xrange(train_x.shape[1]):        
        new_state,myloss,_ = sess.run([out_state,loss,train_op], 
               feed_dict={x_data: train_x[:,i].reshape(-1,1), 
                          y_real:train_y[:,i].reshape(-1,1),
                          c_in:np_c, h_in:np_h
                         })        
        np_c=new_state.c
        np_h=new_state.h
    if j%500 == 0:
        print myloss

4417.588
912.04236
222.94768
125.8858
57.600925
50.0817
74.85818
28.053059
7.297879
21.0089
8.8673115
13.872299
23.86504
2.0798922
2.675952
25.015575
6.814285
83.88816
39.195007
15.484887


Let's check with the known sequences

In [57]:
np_c = np.zeros((1, lstm_size),np.float32)
np_h = np.zeros((1, lstm_size),np.float32)

seq_id=0

out_seq=np.array([])
for i in xrange(test_x.shape[1]):
    out_val,new_state = sess.run([out,out_state], feed_dict={x_data: test_x[seq_id,i].reshape(-1,1),       
                                                             c_in:np_c, h_in:np_h})
    out_seq = np.append(out_seq,out_val)
    np_c=new_state.c
    np_h=new_state.h

print "The estimated output sequence is:"
print out_seq
print "The real output sequence is:"
print test_y[seq_id,:]

The estimated output sequence is:
[12.32107353 22.30460358 43.58792496 42.40033722 61.8417778  53.01939011
 56.33670044 58.39256287 45.43508148 56.55605316 54.44507599 44.94801712
 69.1426239  39.26014709 48.01186752 23.61235046 35.5365715  70.59922028
 53.08533478 25.87182236 52.11916351 55.65919113 37.14069748 41.71841049
 51.17979431 57.79838562 43.18199921 46.7674408  47.27316666 51.6287117 ]
The real output sequence is:
[ 0. 21. 42. 42. 53. 48. 64. 76. 48. 48. 51. 45. 63. 48. 44. 40. 47. 93.
 63. 26. 55. 62. 42. 35. 61. 65. 47. 42. 49. 48.]


## Sequences with N-dimensions

We are going to start creating a series for training.
\\[
\begin{align}
  y[0] &= x_0[0] \\
  y[1] &= x_0[1] \\
  y[2] &= x_0[0]+x_0[1]+x_1[2]+x_1[1]    
\end{align}
\\]



In [3]:
num_seqs=7
seq_len=4
seqs_x=np.array([]).reshape(-1,seq_len,2)
seqs_y=np.array([]).reshape(-1,seq_len)
for i in xrange(num_seqs):
    x0=np.empty((0),dtype=np.float64)
    x1=np.empty((0),dtype=np.float64)    
    for j in xrange(seq_len):
        x0=np.append(x0,np.float64(np.random.randint(0,50)))
        x1=np.append(x1,np.float64(np.random.randint(0,50)))

    y=np.empty_like(x0)
    y[0]=x0[0]
    y[1]=x0[1]
    for j in xrange(2,seq_len):
        y[j]=x0[j]+x0[j-1]+x1[j]
    x_vec=np.concatenate((np.reshape(x0,(1,-1,1)),np.reshape(x1,(1,-1,1))),axis=2)
    seqs_x=np.concatenate((seqs_x, x_vec ),axis=0)
    seqs_y=np.concatenate((seqs_y, np.reshape(y,(1,-1))))
    
    

print "Sequences X:"
print seqs_x
print "Sequences Y:"
print seqs_y

Sequences X:
[[[20. 17.]
  [ 2. 47.]
  [22. 15.]
  [38. 12.]]

 [[28. 38.]
  [32. 24.]
  [27. 19.]
  [14.  1.]]

 [[49.  3.]
  [39.  8.]
  [41. 13.]
  [ 9.  8.]]

 [[34. 10.]
  [27.  6.]
  [ 3. 35.]
  [23. 24.]]

 [[21. 26.]
  [34.  0.]
  [20. 42.]
  [13. 20.]]

 [[49. 33.]
  [ 7. 20.]
  [22.  4.]
  [41. 46.]]

 [[ 8. 49.]
  [32.  4.]
  [20.  2.]
  [37. 37.]]]
Sequences Y:
[[ 20.   2.  39.  72.]
 [ 28.  32.  78.  42.]
 [ 49.  39.  93.  58.]
 [ 34.  27.  65.  50.]
 [ 21.  34.  96.  53.]
 [ 49.   7.  33. 109.]
 [  8.  32.  54.  94.]]


In [12]:
th = np.int(num_seqs*0.7)
train_x= seqs_x[:th]
test_x = seqs_x[th:]
train_y= seqs_y[:th]
test_y = seqs_y[th:]

In [13]:
lstm_size = 10
num_features = 1

In [14]:
tf.reset_default_graph()
num_features=2

#The input data is going to be [batch_size]
x_data = tf.placeholder(tf.float32, [None, num_features],"x_data")

#The output data is going to be [batch_size, 1]
y_real = tf.placeholder(tf.float32, [None, 1],"y_real")

lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size, state_is_tuple=True,name="LSTM_cell")
c_in=tf.placeholder(tf.float32, [None, lstm_size],name="c_in")
h_in=tf.placeholder(tf.float32, [None, lstm_size],name="h_in")
                    
init_state = tf.contrib.rnn.LSTMStateTuple( c=c_in, h=h_in)


# The value of state is updated after processing each batch of words.
output_rnn, out_state = lstm(x_data, init_state)

W_out = tf.get_variable("Wout",initializer=tf.random_normal([lstm_size,1]))
b_out = tf.get_variable("b_out",initializer=tf.random_normal([1]))

# The LSTM output can be used to make next word predictions
out = tf.matmul(output_rnn, W_out) + b_out

loss = tf.losses.mean_squared_error(y_real,out)
#train_op = tf.train.AdamOptimizer(learning_rate=0.0002).minimize(loss)

optimizer = tf.train.AdamOptimizer(learning_rate=0.0002)
gvs = optimizer.compute_gradients(loss)
capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
train_op = optimizer.apply_gradients(capped_gvs)

sess=tf.Session()
sess.run(tf.global_variables_initializer())

# We save the graph to analyze it in Tensorboard
train_writer = tf.summary.FileWriter('./graphtf',sess.graph)

Le'ts try with an example, yust one sample from the list

In [22]:
np_c = np.zeros((1, lstm_size),np.float32)
np_h = np.zeros((1, lstm_size),np.float32)

o = sess.run([out], feed_dict={x_data: train_x[0:1,0], 
                                      y_real:train_y[0:1,0].reshape(-1,1),
                                      c_in:np_c, h_in:np_h})

print "The estimated output without training is:"
print np.squeeze(o)
print "But it should be:"
print train_y[1,0]

The estimated output without training is:
-0.2244373
But it should be:
28.0


Let's train the model

In [24]:
max_epocs=10000
for j in xrange(max_epocs):
    np_c = np.zeros((train_x.shape[0], lstm_size),np.float32)
    np_h = np.zeros((train_x.shape[0], lstm_size),np.float32)
    for i in xrange(train_x.shape[1]):        
        new_state,myloss,_ = sess.run([out_state,loss,train_op], 
               feed_dict={x_data: train_x[:,i], 
                          y_real:train_y[:,i].reshape(-1,1),
                          c_in:np_c, h_in:np_h
                         })        
        np_c=new_state.c
        np_h=new_state.h
    if j%500 == 0:
        print myloss

91.882385
86.09121
162.65552
145.52869
136.37135
115.608345
95.841606
80.55209
72.08536
45.366634
59.180256
37.831577
24.193466
98.015366
26.17407
26.645082
13.29495
3.6439674
8.933705
8.963353


In [28]:
test_x[1,1,:].shape

(2,)

In [29]:
np_c = np.zeros((1, lstm_size),np.float32)
np_h = np.zeros((1, lstm_size),np.float32)

seq_id=0

out_seq=np.array([])
for i in xrange(test_x.shape[1]):
    out_val,new_state = sess.run([out,out_state], feed_dict={x_data: test_x[seq_id,i].reshape(1,2),       
                                                             c_in:np_c, h_in:np_h})
    out_seq = np.append(out_seq,out_val)
    np_c=new_state.c
    np_h=new_state.h

print "The estimated output sequence is:"
print out_seq
print "The real output sequence is:"
print test_y[seq_id,:]

The estimated output sequence is:
[22.26746559 64.88938904 64.10012817 58.24990845]
The real output sequence is:
[21. 34. 96. 53.]


## Train more than one timestep at a time


https://www.tensorflow.org/api_docs/python/tf/nn/static_rnn

https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn

In [109]:
test_y[seq_id,:]

array([29., 44., 73., 74., 47., 45., 29., 40., 48., 52., 90., 52., 42.,
       84., 92.])

In [17]:
train_x = np.array([[[1], [2], [5], [6]],
          [[5], [7], [7], [8]],
          [[3], [4], [5], [7]]])

train_x = np.array([[[1], [2], [5], [6]],
          [[5], [7], [7], [8]],
          [[3], [4], [5], [7]],
          [[2], [3], [7], [8]]])

train_y = np.array([[1, 3, 7, 11],
           [5, 12, 14, 15],
           [3, 7, 9, 12],
           [2, 5, 10 ,15]])

test_x = np.array([[[1], [2], [3], [4]],
          [[4], [5], [6], [7]]])

In [3]:
time_steps = 4
num_features = 1
lstm_size=10

In [18]:
tf.reset_default_graph()


batch_step = tf.placeholder(tf.float32, [None, num_features],"x_train")
y_real = tf.placeholder(tf.float32, [None, num_features])

batch_size = tf.shape(batch_step)[0]
lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size, state_is_tuple=True,name="LSTM_cell")
# Initial state of the LSTM memory.
#init_state = state = lstm.zero_state(batch_size,tf.float32)
#c_in=tf.zeros((batch_size, lstm_size), np.float32,name="c_in")
#h_in=tf.zeros((batch_size, lstm_size),np.float32,name="h_in")
c_in=tf.placeholder(tf.float32, [None, lstm_size],name="c_in")
h_in=tf.placeholder(tf.float32, [None, lstm_size],name="h_in")
                    
init_state = tf.contrib.rnn.LSTMStateTuple( c=c_in, h=h_in)
#reset_state=tf.identity_n(init_state)


loss = 0.0
# The value of state is updated after processing each batch of words.
output_rnn, out_state = lstm(batch_step, init_state)


W_out = tf.get_variable("Wout",initializer=tf.random_normal([lstm_size,1]))
b_out = tf.get_variable("b_out",initializer=tf.random_normal([1]))


# The LSTM output can be used to make next word predictions
out = tf.matmul(output_rnn, W_out) + b_out

loss = tf.losses.mean_squared_error(y_real,out)
train_op = tf.train.AdamOptimizer().minimize(loss)

sess=tf.Session()
sess.run(tf.global_variables_initializer())

np_c = np.zeros((train_x.shape[0], lstm_size),np.float32)
np_h = np.zeros((train_x.shape[0], lstm_size),np.float32)

print sess.run([out,loss,train_op], 
               feed_dict={batch_step: train_x[:,0,:], 
                          y_real:train_y[:,0].reshape(-1,1),
                          c_in:np_c, h_in:np_h
                         })
train_writer = tf.summary.FileWriter('./graphtf',sess.graph)


[array([[-0.950649  ],
       [-0.6088793 ],
       [-0.75499594],
       [-0.84744596]], dtype=float32), 14.368125, None]


In [21]:
max_epocs=1000
for j in xrange(max_epocs):
    np_c = np.zeros((train_x.shape[0], lstm_size),np.float32)
    np_h = np.zeros((train_x.shape[0], lstm_size),np.float32)
    for i in xrange(4):        
        new_state,myloss,_ = sess.run([out_state,loss,train_op], 
               feed_dict={batch_step: train_x[:,i,:], 
                          y_real:train_y[:,i].reshape(-1,1),
                          c_in:np_c, h_in:np_h
                         })        
        np_c=new_state.c
        np_h=new_state.h
    if j%100 == 0:
        print myloss

0.49185872
0.40784678
0.34643236
0.30182305
0.27053022
0.24956086
0.23522538
0.2237002
0.21229109
0.19987653


In [23]:
np_c = np.zeros((test_x.shape[0], lstm_size),np.float32)
np_h = np.zeros((test_x.shape[0], lstm_size),np.float32)

for i in xrange(4):
    out_val,new_state = sess.run([out,out_state], feed_dict={batch_step: test_x[:,i,:],c_in:np_c, h_in:np_h})
    np_c=new_state.c
    np_h=new_state.h
    print out_val
    

[[0.9709455]
 [4.0639324]]
[[2.9082494]
 [9.049345 ]]
[[ 4.6729946]
 [11.843623 ]]
[[ 6.1187854]
 [12.876646 ]]


In [428]:
test_x

array([[[1],
        [2],
        [3],
        [4]],

       [[4],
        [5],
        [6],
        [7]]])

In [ ]:
[[ 1.6971655  2.9445994  4.9434905  7.257107 ]
 [ 4.436798   9.061197  11.678692  12.575489 ]]


https://medium.com/@erikhallstrm/hello-world-rnn-83cd7105b767

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn

In [37]:
class SeriesPredictor:
    def __init__(self, input_dim, seq_size, hidden_dim=10):
        
        self.input_dim=input_dim
        self.seq_size=seq_size
        self.hidden_dim=hidden_dim
        
        self.W_out = tf.Variable(tf.random_normal([hidden_dim,1]),name='W_out')
        self.b_out = tf.Variable(tf.random_normal([1]),name='b_out')
        self.x = tf.placeholder(tf.float32, [None, seq_size, input_dim])
        self.y = tf.placeholder(tf.float32, [None, seq_size])
        
        
        self.cost = tf.reduce_mean(tf.square(self.model()-self.y))
        self.train_op = tf.train.AdamOptimizer().minimize(self.cost)
        
        self.saver = tf.train.Saver()
        
        
    def model(self):
        """
        :param x: inputs of size [T, batch_size, input_size]
        :param W: matrix of fully-connected output layer weights
        :param b: vector of fully-connected output layer biases
        """
        cell = rnn.BasicLSTMCell(self.hidden_dim)
        outputs, states = tf.nn.dynamic_rnn(cell, self.x, dtype=tf.float32)
        num_examples = tf.shape(self.x)[0]
        W_repeated = tf.tile(tf.expand_dims(self.W_out,0), [num_examples,1,1])
        out = tf.matmul(outputs, W_repeated)+self.b_out
        out = tf.squeeze(out)
        return out
    
    def train(self, train_x, train_y):
        with tf.Session() as sess:
            tf.get_variable_scope().reuse_variables()
            sess.run(tf.global_variables_initializer())
            for i in range(1000):
                _, mse = sess.run([self.train_op, self.cost], feed_dict={self.x: train_x, self.y: train_y})
                if i% 100 == 0:
                    print(i,mse)
            save_path = self.saver.save(sess,'./model.ckpt')
            print('Model saved to {}'.format(save_path))
                
        
    def test(self,test_x):
        with tf.Session() as sess:
            tf.get_variable_scope().reuse_variables()
            self.saver.restore(sess, './model.ckpt')
            output = sess.run(self.model(), feed_dict={self.x: test_x})
            print(output)

    
        

In [10]:
class SeriesPredictor:
    def __init__(self, input_dim, seq_size, hidden_dim=10):
        
        self.input_dim=input_dim
        self.seq_size=seq_size
        self.hidden_dim=hidden_dim
        
        self.W_out = tf.get_variable("Wout",initializer=tf.random_normal([hidden_dim,1]))
        self.b_out = tf.get_variable("b_out",initializer=tf.random_normal([1]))
        self.x = tf.placeholder(tf.float32, [None, seq_size, input_dim])
        self.y = tf.placeholder(tf.float32, [None, seq_size])
        
        self.mymodel = self.model()
        self.cost = tf.reduce_mean(tf.square(self.mymodel-self.y))
        self.train_op = tf.train.AdamOptimizer().minimize(self.cost)
        
        self.saver = tf.train.Saver()
        
    def model(self):
        """
        :param x: inputs of size [T, batch_size, input_size]
        :param W: matrix of fully-connected output layer weights
        :param b: vector of fully-connected output layer biases
        """
        cell = rnn.BasicLSTMCell(self.hidden_dim)
        outputs, states = tf.nn.dynamic_rnn(cell, self.x, dtype=tf.float32)
        num_examples = tf.shape(self.x)[0]
        W_repeated = tf.tile(tf.expand_dims(self.W_out,0), [num_examples,1,1])
        out = tf.matmul(outputs, W_repeated)+self.b_out
        out = tf.squeeze(out)
        return out
    
    def train(self, train_x, train_y):
        sess = tf.Session()
        sess.run(tf.global_variables_initializer())
        for i in range(1000):
            _, mse = sess.run([self.train_op, self.cost], feed_dict={self.x: train_x, self.y: train_y})
            if i% 100 == 0:
                print(i,mse)
        return sess
                
        
    def test(self,sess,test_x):        
        output = sess.run(self.mymodel, feed_dict={self.x: test_x})
        print(output)

    
        

In [11]:
tf.reset_default_graph()
predictor = SeriesPredictor(input_dim=1, seq_size=4, hidden_dim=10)
train_x = [[[1], [2], [5], [6]],
          [[5], [7], [7], [8]],
          [[3], [4], [5], [7]]]
train_y = [[1, 3, 7, 11],
           [5, 12, 14, 15],
           [3, 7, 9, 12]]
sess=predictor.train(train_x, train_y)

test_x = [[[1], [2], [3], [4]],
          [[4], [5], [6], [7]]]

predictor.test(sess,test_x)

(0, 104.54908)
(100, 49.596928)
(200, 13.441304)
(300, 4.84034)
(400, 3.223334)
(500, 2.582983)
(600, 1.9891154)
(700, 1.4041783)
(800, 1.0117636)
(900, 0.73919386)
[[ 0.69609344  2.6687462   5.052478    6.9160995 ]
 [ 4.0849714   9.2390785  11.849617   12.85392   ]]


In [17]:
train_x[0][0]

[[1], [2], [5], [6]]